In [ ]:
# %matplotlib qt
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ExifTags

#이미지 가져오기

#필요시 이미지 회전
def rotate_image_based_on_exif(image_path):
    img = Image.open(image_path)

    try:
        # 이미지의 EXIF 정보를 가져옴
        exif = img._getexif()
        if exif is not None:
            # 모든 EXIF 태그 중에서 'Orientation' 태그 찾기
            for tag, value in ExifTags.TAGS.items():
                if value == 'Orientation':
                    orientation = exif.get(tag)
                    break

            # 이미지 회전 (필요한 경우)
            if orientation == 3:  # 180도 회전
                img = img.rotate(180, expand=True)
            elif orientation == 6:  # 시계방향으로 270도 회전 (90도 반시계)
                img = img.rotate(270, expand=True)
            elif orientation == 8:  # 시계방향으로 90도 회전 (270도 반시계)
                img = img.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # EXIF 정보가 없거나, 태그가 없는 경우
        pass

    return img

img_folder = 'image'
img_name_list = os.listdir(img_folder)
img_list=[]

fig = plt.figure("images")

num = 0
for i in img_name_list:
    
    img_list.append(np.array(rotate_image_based_on_exif(img_folder+'/'+i)))
    ax = fig.add_subplot(1,2,num+1)
    ax.axis('off')
    ax.imshow(img_list[num])
    num+=1
    

# 레이아웃 조정
plt.tight_layout()
plt.show()    


In [ ]:

#특징점 만들기

#특징점 검출기

def feature_detector(feature_name):
    if feature_name=='SIFT':
        detector = cv2.SIFT_create()
        print("SIFT")
    elif feature_name=='ORB':
        detector = cv2.ORB_create()
        print("ORB")
    elif feature_name=='BRISK':
        detector = cv2.BRISK_create()
        print("BRISK")
    else:
        print('SIFT,ORB,BRISK 중 하나를 입력')


    return detector

feature_name = 'SIFT'
detector = feature_detector(feature_name)

keypoint_list = []
descriptor_list = []
num=0
fig = plt.figure(feature_name)
for i in img_list:
    gray = cv2.cvtColor(i,cv2.COLOR_BGR2GRAY)
    keypoint, descriptor = detector.detectAndCompute(gray,None)
    keypoint_list.append(keypoint)
    descriptor_list.append(descriptor)
    keypoints_in_img = cv2.drawKeypoints(i,keypoint,outImage=None,color=(0,0,255) ,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    ax = fig.add_subplot(1,2,num+1)
    ax.axis('off')
    ax.imshow(keypoints_in_img)
    num+=1

# 레이아웃 조정
plt.tight_layout()
plt.show()  
#특징점 이미지에 그리기
